In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_from_disk

new_ds = load_from_disk("/content/drive/MyDrive/LHL/llm-project/new_ds_v2")

In [ ]:
from transformers import pipeline

In [ ]:
# Tasks summary https://huggingface.co/docs/transformers/main/en/task_summary
pipe = pipeline(task='sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
print(new_ds["train"].column_names)

['artist', 'album', 'genre', 'author', 'score', 'review', 'augmented_review', 'label']


In [ ]:
data = new_ds["train"]["review"]
print(len(data))  # Check the number of reviews

19305


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load DistilBERT model for binary classification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english",
    num_labels=2  # Binary classification (0 = Negative, 1 = Positive)
)

# Check model structure
print(model)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from transformers import TrainingArguments

# Define training parameters without W&B
training_args = TrainingArguments(
    output_dir="./distilbert_finetuned_album_reviews",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=4,  # Keep batch size low for efficiency
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # ✅ Increase to 3 epochs
    learning_rate=5e-5,  # Slightly higher learning rate for quicker adaptation
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Print training arguments to confirm
print(training_args)

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import random
from datasets import Dataset

# Separate positive and negative reviews
positive_reviews = [i for i, label in enumerate(new_ds["train"]["label"]) if label == 1]
negative_reviews = [i for i, label in enumerate(new_ds["train"]["label"]) if label == 0]

# Randomly select 100 from each
random.seed(42)
sampled_positives = random.sample(positive_reviews, 100)
sampled_negatives = random.sample(negative_reviews, 100)

# Combine selected reviews
sample_indices = sampled_positives + sampled_negatives
random.shuffle(sample_indices)  # Shuffle to mix labels

# Create a new smaller dataset
medium_train_data = Dataset.from_dict({
    "text": [new_ds["train"]["review"][i] for i in sample_indices],
    "label": [new_ds["train"]["label"][i] for i in sample_indices],
})

# Check dataset size
print(medium_train_data)

Dataset({
    features: ['text', 'label'],
    num_rows: 200
})


In [ ]:
from transformers import AutoTokenizer

# Load DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Now, apply tokenization
tokenized_medium_dataset = medium_train_data.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_medium_dataset.column_names)

['text', 'label', 'input_ids', 'attention_mask']


In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
from transformers import Trainer

# Define a function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convert logits to label predictions
    return {"accuracy": accuracy_score(labels, predictions)}

# Reload Trainer with evaluation metric
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_medium_dataset,  # ✅ Use 200-review dataset
    eval_dataset=tokenized_medium_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-14-cb5925c314ce>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Run trainer on medium dataset
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.870600,0.581219,0.555000
2,0.470800,0.150635,0.945000
3,0.095700,0.036120,0.985000


TrainOutput(global_step=150, training_loss=0.47904631932576497, metrics={'train_runtime': 2742.0634, 'train_samples_per_second': 0.219, 'train_steps_per_second': 0.055, 'total_flos': 79480439193600.0, 'train_loss': 0.47904631932576497, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/LHL/llm-project/untuned_distilbert_model")
tokenizer.save_pretrained("/content/drive/MyDrive/LHL/llm-project/untuned_distilbert_model")


('/content/drive/MyDrive/LHL/llm-project/untuned_distilbert_model/tokenizer_config.json',
 '/content/drive/MyDrive/LHL/llm-project/untuned_distilbert_model/special_tokens_map.json',
 '/content/drive/MyDrive/LHL/llm-project/untuned_distilbert_model/vocab.txt',
 '/content/drive/MyDrive/LHL/llm-project/untuned_distilbert_model/added_tokens.json',
 '/content/drive/MyDrive/LHL/llm-project/untuned_distilbert_model/tokenizer.json')

In [ ]:
import random
from datasets import Dataset

# Separate positive and negative reviews
positive_reviews = [i for i, label in enumerate(new_ds["test"]["label"]) if label == 1]
negative_reviews = [i for i, label in enumerate(new_ds["test"]["label"]) if label == 0]

# Randomly select 100 from each
random.seed(42)
sampled_test_positives = random.sample(positive_reviews, 100)
sampled_test_negatives = random.sample(negative_reviews, 100)

# Combine selected reviews
test_sample_indices = sampled_test_positives + sampled_test_negatives
random.shuffle(test_sample_indices)  # Shuffle to mix labels

# Create a new test dataset
test_data = Dataset.from_dict({
    "text": [new_ds["test"]["review"][i] for i in test_sample_indices],  # ✅ FIX: Using "test" instead of "train"
    "label": [new_ds["test"]["label"][i] for i in test_sample_indices],
})

# Check dataset size
print(test_data)

Dataset({
    features: ['text', 'label'],
    num_rows: 200
})


In [ ]:
# Tokenize the test dataset
tokenized_test_data = test_data.map(tokenize_function, batched=True)

print(tokenized_test_data.column_names)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

['text', 'label', 'input_ids', 'attention_mask']


In [ ]:
# Get predictions on the test set
test_predictions = trainer.predict(tokenized_test_data)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Compute accuracy function (needed for evaluation)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convert logits to label predictions
    return {"accuracy": accuracy_score(labels, predictions)}

# Get predictions on the test set
test_predictions = trainer.predict(tokenized_test_data)

# Convert logits to label predictions (0 = Negative, 1 = Positive)
predicted_labels = np.argmax(test_predictions.predictions, axis=-1)

# Extract actual labels
true_labels = test_data["label"]

# Compute accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.6400


In [ ]:
# Define tokenization function using "review" instead of "text"
def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization to the full dataset
tokenized_full_data = new_ds["train"].map(tokenize_function, batched=True)

# Check that tokenization was successful
print(tokenized_full_data.column_names)

['artist', 'album', 'genre', 'author', 'score', 'review', 'augmented_review', 'label', 'input_ids', 'attention_mask']


In [ ]:
# Adjust for CPU processing
training_args = TrainingArguments(
    output_dir="./distilbert_finetuned_album_reviews",
    per_device_eval_batch_size=4,  # ✅ Smaller batch size to prevent slowdowns
    report_to="none"
)

# Reload Trainer with CPU-friendly settings
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer
)

# Run predictions on CPU
full_predictions = trainer.predict(tokenized_full_data)

<ipython-input-22-964b922ef84f>:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import numpy as np

# Regenerate predicted labels
predicted_labels = np.argmax(full_predictions.predictions, axis=-1)

# Check if the lengths match now
print(len(new_ds["train"]["artist"]))
print(len(predicted_labels))

19305
19305


In [ ]:
import pandas as pd

df_predictions = pd.DataFrame({
    "artist": new_ds["train"]["artist"],
    "album": new_ds["train"]["album"],
    "score": new_ds["train"]["score"],
    "actual_label": new_ds["train"]["label"],  # Ground truth
    "predicted_label": predicted_labels  # Model predictions
})

# Display first 10 rows
df_predictions.head(10)

,artist,album,score,actual_label,predicted_label
0,Gold Chains,Young Miss America,6.7,0,1
1,Sunn O))) & Boris,Altar,6.6,0,1
2,Sloan,A Sides Win: Singles 1992-2005,7.9,1,0
3,The Antlers,Hospice,8.5,1,0
4,Krieg,Transient,8.0,1,1
5,Mndsgn,Body Wash,7.7,1,1
6,The Strokes,Future Present Past EP,6.0,0,0
7,Eminem,Recovery,2.8,0,0
8,UNKLE,Never Never Land,5.0,0,0
9,Chuck Johnson,Balsams,8.1,1,1


In [ ]:
from sklearn.metrics import accuracy_score

# Compute final accuracy
final_accuracy = accuracy_score(new_ds["train"]["label"], predicted_labels)

# Display accuracy score
print(f"Final Accuracy: {final_accuracy:.4f}")

Final Accuracy: 0.6072


In [ ]:
model.save_pretrained("/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model")
tokenizer.save_pretrained("/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model")

('/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model/tokenizer_config.json',
 '/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model/special_tokens_map.json',
 '/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model/vocab.txt',
 '/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model/added_tokens.json',
 '/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model/tokenizer.json')

In [ ]:
new_ds.save_to_disk("/content/drive/MyDrive/LHL/llm-project/new_ds_with_distilbert_predictions")

Saving the dataset (0/1 shards):   0%|          | 0/19305 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1566 [00:00<?, ? examples/s]

In [ ]:
# reloading model
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model"
)
tokenizer = AutoTokenizer.from_pretrained(
    "/content/drive/MyDrive/LHL/llm-project/lightly_tuned_distilbert_model"
)


In [ ]:
# reloading data
from datasets import load_from_disk

new_ds = load_from_disk("/content/drive/MyDrive/LHL/llm-project/new_ds_with_distilbert_predictions")
